## 10.5 多头注意力

也就是将设计多个注意力模块并联，学习多几次，下图展示了多头注意力的样子。

![](../images/10-5-1.svg)

### 10.5.1 模型

对于给定查询$\bm{q}\in\mathbb{R}^{d_q}$、键$\bm{k}\in\mathbb{R}^{d_k}$和值$\bm{v}\in\mathbb{R}^{d_v}$而言，每个注意力头$\bm{h}_i\quad (i=1,2,\dots,h)$的计算方法为：

$$
\bm{h}_i=f(\bm{W}_i^{(q)}\bm{q},\bm{W}_i^{(k)}\bm{k},\bm{W}_i^{(v)}\bm{v})\in\mathbb{R}^{p_v}
$$

这里的还有一个可学习的参数$\bm{W}_o\in\mathbb{R}^{p_o\times hp_v}$，其作用于$h$个头连接之后的结果。

$$
\bm{W}_o\begin{bmatrix}\bm{h}_1\\\vdots\\\bm{h}_h\end{bmatrix}
$$

这种稍微复杂的设计可以使得每个头关注不同的部分。

In [1]:
import math
import torch
from torch import nn
from d2l import torch as d2l

### 10.5.2 实现

通常选择缩放点积注意力作为每一个注意力头。为了避免计算代价和参数代价的大幅增长，设定$p_q=p_k=p_v=p_o/h$。在下面的实现中$p_o$中的参数是由`num_hiddens`来指定的。

在这里，`num_hiddens`和`num_heads`的配合共同实现了多头注意力，可以这样理解，这里的`num_hiddens`不是真正的每个头里的隐层，而是所有头的隐层的和，也就是每个头里的隐层的数量为`num_hiddens/num_heads`。

好吧实际上，可以把下述的代码理解为一个巨大的单头注意力，也就是进行了并行优化的多头注意力。

In [2]:
#@save
class MultiHeadAttention(nn.Module):
    """多头注意力"""
    def __init__(self, key_size, query_size, value_size, num_hiddens,
                 num_heads, dropout, bias=False, **kwargs):
        super(MultiHeadAttention, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.attention = d2l.DotProductAttention(dropout)
        self.W_q = nn.Linear(query_size, num_hiddens, bias=bias)
        self.W_k = nn.Linear(key_size, num_hiddens, bias=bias)
        self.W_v = nn.Linear(value_size, num_hiddens, bias=bias)
        self.W_o = nn.Linear(num_hiddens, num_hiddens, bias=bias)
    
    def forward(self, queries, keys, values, valid_lens):
        """
        queries, keys, values的相同形状：
        (batch_size, 查询或键值对的个数, num_hiddens) \\
        valid_lens的形状:
        (batch_size, )或(batch_size, 查询的个数) \\
        经过变换后，输出的queries, keys, values的形状：
        (batch_size*num_heads, 查询或键值对的个数, num_hiddens/num_heads)
        """
        queries = transpose_qkv(self.W_q(queries), self.num_heads)
        keys = transpose_qkv(self.W_k(keys), self.num_heads)
        values = transpose_qkv(self.W_v(values), self.num_heads)

        if valid_lens is not None:
            # 在轴0，将第一项（标量或者矢量）复制num_heads次，
            # 然后如此复制第二项，然后诸如此类。
            valid_lens = torch.repeat_interleave(
                valid_lens, repeats=self.num_heads, dim=0)
        
        # output.shape:
        # (batch_size*num_heads, 查询的个数, num_hiddens/num_heads)
        output = self.attention(queries, keys, values, valid_lens)

        # output_concat.shape:
        # (batch_size, 查询的个数, num_hiddens)
        output_concat = transpose_output(output, self.num_heads)
        return self.W_o(output_concat)

为了进行并行计算，需要定义下面两个函数。

In [3]:
#@save
def transpose_qkv(X, num_heads):
    """
    为了多头注意力计算而改变形状

    shapes:
    input X: (batch_size, 查询或键值对的个数, num_hiddens)
    output X: (batch_size*num_heads, 查询或键值对的个数, num_hiddens/num_heads)
    """
    X = X.reshape(X.shape[0], X.shape[1], num_heads, -1)
    X = X.permute(0, 2, 1, 3)
    return X.reshape(-1, X.shape[2], X.shape[3])

#@save
def transpose_output(X, num_heads):
    """逆转transpose_qkv函数的操作"""
    X = X.reshape(-1, num_heads, X.shape[1], X.shape[2])
    X = X.permute(0, 2, 1, 3)
    return X.reshape(X.shape[0], X.shape[1], -1)

下面来测试一下，这里的键和值是相同的，多头注意力输出的形状是`(batch_size, num_queries, num_hiddens)`。

In [4]:
num_hiddens, num_heads = 100, 5
attention = MultiHeadAttention(
    key_size=num_hiddens,
    query_size=num_hiddens,
    value_size=num_hiddens,
    num_heads=num_heads,
    num_hiddens=num_hiddens,
    dropout=0.5,
)
attention.eval()

MultiHeadAttention(
  (attention): DotProductAttention(
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (W_q): Linear(in_features=100, out_features=100, bias=False)
  (W_k): Linear(in_features=100, out_features=100, bias=False)
  (W_v): Linear(in_features=100, out_features=100, bias=False)
  (W_o): Linear(in_features=100, out_features=100, bias=False)
)

下面制造一点qkv

In [5]:
batch_size, num_queries = 2, 4
num_kvpairs, valid_lens = 6, torch.tensor([3, 2])
X = torch.ones((batch_size, num_queries, num_hiddens))
Y = torch.ones((batch_size, num_kvpairs, num_hiddens))
attention(X, Y, Y, valid_lens).shape

torch.Size([2, 4, 100])

上述的例子不够通用，我们使用自己的例子来示范。

在这里，虽然使用了缩放点积注意力，但是由于QKV需要处理为相同的`num_hiddens`在送入注意力中，所以也没有查询的特征维度和键的特征维度相同的需求了。

In [6]:
query_hiddens = 5
key_hiddens = 6
value_hiddens = 7
Q = torch.rand((batch_size, num_queries, query_hiddens))
K = torch.rand((batch_size, num_kvpairs, key_hiddens))
V = torch.rand((batch_size, num_kvpairs, value_hiddens))
attention = MultiHeadAttention(
    query_size=query_hiddens,
    key_size=key_hiddens,
    value_size=value_hiddens,
    num_heads=num_heads,
    num_hiddens=num_hiddens,
    dropout=0.5
)
attention(Q, K, V, valid_lens).shape

torch.Size([2, 4, 100])

### 10.5.3 小结

- 多头注意力融合了来自于多个注意力汇聚的不同知识，这些知识的不同来源于相同的查询、键和值的不同的子空间表示。
- 基于适当的张量操作，可以实现多头注意力的并行计算。